In [64]:
#pip install pytorch_forecasting

In [65]:
import pandas as pd
from pytorch_forecasting import  TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer

In [76]:
#Open demand forcasting data currently set to one day series
data = pd.read_csv("/notebooks/pytorch_forecasting.csv")

In [77]:
#Check top 5 rows
data.head()

,STATE,DATETIME,TOTALDEMAND,POWER,LOCATION,TEMPERATURE,FORECASTDEMAND,Weekday,Quarter,Month,Season,Day,Year,SUNRISE,SUNSET,DAYTYPE,HOLIDAY,time_idx
0,NSW,2018-03-07 00:00:00,7183.02,0.0,Bankstown,19.3,7183.852254,Wednesday,1,3,Autumn,7,2018,2018-03-07 05:49:00,2018-03-07 18:25:00,Night,False,0
1,NSW,2018-03-07 00:30:00,7005.89,0.0,Bankstown,19.0,7030.936389,Wednesday,1,3,Autumn,7,2018,2018-03-07 05:49:00,2018-03-07 18:25:00,Night,False,1
2,NSW,2018-03-07 01:00:00,6763.68,0.0,Bankstown,18.0,6842.411918,Wednesday,1,3,Autumn,7,2018,2018-03-07 05:49:00,2018-03-07 18:25:00,Night,False,2
3,NSW,2018-03-07 01:30:00,6510.93,0.0,Bankstown,17.5,6550.943514,Wednesday,1,3,Autumn,7,2018,2018-03-07 05:49:00,2018-03-07 18:25:00,Night,False,3
4,NSW,2018-03-07 02:00:00,6273.88,0.0,Bankstown,17.5,6318.996800,Wednesday,1,3,Autumn,7,2018,2018-03-07 05:49:00,2018-03-07 18:25:00,Night,False,4


In [78]:
# Create a hour index
time_idx = {
        '00:00:00': 0,
        '00:30:00': 1,
        '01:00:00': 2,
        '01:30:00': 3,
        '02:00:00': 4,
        '02:30:00': 5,
        '03:00:00': 6,
        '03:30:00': 7,
        '04:00:00': 8,
        '04:30:00': 9,
        '05:00:00': 10,
        '05:30:00': 11,
        '06:00:00': 12,
        '06:30:00': 13,
        '07:00:00': 14,
        '07:30:00': 15,
        '08:00:00': 16,
        '08:30:00': 17,
        '09:00:00': 18,
        '09:30:00': 19,
        '10:00:00': 20,
        '10:30:00': 21,
        '11:00:00': 22,
        '11:30:00': 23,
        '12:00:00': 24,
        '12:30:00': 25,
        '13:00:00': 26,
        '13:30:00': 27,
        '14:00:00': 28,
        '14:30:00': 29,
        '15:00:00': 30,
        '15:30:00': 31,
        '16:00:00': 32,
        '16:30:00': 33,
        '17:00:00': 34,
        '17:30:00': 35,
        '18:00:00': 36,
        '18:30:00': 37,
        '19:00:00': 38,
        '19:30:00': 39,
        '20:00:00': 40,
        '20:30:00': 41,
        '21:00:00': 42,
        '21:30:00': 43,
        '22:00:00': 44,
        '22:30:00': 45,
        '23:00:00': 46,
        '23:30:00': 47,
    }
def gen_time_idx(time):
    return time_idx[str(time)]

In [79]:
#create a day index
day_idx = {
    1:0,
    2:48,
    3:96,
    4:144,
    5:192,
    6:240,
    7:288,
    8:336,
    9:384,
    10:432,
    11:480,
    12:528,
    13:576,
    14:624,
    15:672,
    16:720,
    17:768,
    18:816,
    19:864,
    20:912,
    21:960,
    22:1008,
    23:1056,
    24:1104,
    25:1152,
    26:1200,
    27:1248,
    28:1296,
    29:1344,
    30:1392,
    31:1440
}

In [80]:
def gen_month_idx(date):
    #Get starting index for day
    start = gen_time_idx(date.time())
    day = day_idx[date.day]
    return day + start

In [81]:
data["DATETIME"] = pd.to_datetime(data['DATETIME'])
data["month_idx"] = data['DATETIME'].apply(gen_month_idx)


In [82]:
#Save to csv
data.to_csv("demand_pv_temp_forecast_timefeatures_hols_v2_nsw_month_series.csv", index=False)

In [83]:
data.drop(['STATE', 'LOCATION', 'FORECASTDEMAND', 'SUNRISE', 'SUNSET'], axis=1, inplace=True)
booleanDictionary = {True: 'TRUE', False: 'FALSE'}
data.replace(booleanDictionary, inplace=True)
data.sample(10, random_state=48)
data.head()

,DATETIME,TOTALDEMAND,POWER,TEMPERATURE,Weekday,Quarter,Month,Season,Day,Year,DAYTYPE,HOLIDAY,time_idx,month_idx
0,2018-03-07 00:00:00,7183.02,0.0,19.3,Wednesday,1,3,Autumn,7,2018,Night,FALSE,0,288
1,2018-03-07 00:30:00,7005.89,0.0,19.0,Wednesday,1,3,Autumn,7,2018,Night,FALSE,1,289
2,2018-03-07 01:00:00,6763.68,0.0,18.0,Wednesday,1,3,Autumn,7,2018,Night,FALSE,2,290
3,2018-03-07 01:30:00,6510.93,0.0,17.5,Wednesday,1,3,Autumn,7,2018,Night,FALSE,3,291
4,2018-03-07 02:00:00,6273.88,0.0,17.5,Wednesday,1,3,Autumn,7,2018,Night,FALSE,4,292


In [74]:
# Test creating TimeSeriesDataSet
max_prediction_length = 48
max_encoder_length = 48
training_cutoff = data["month_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="month_idx",
    target="TOTALDEMAND",
    group_ids=["Month", "Year"],
    min_encoder_length= max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=["Weekday", "Season", "DAYTYPE", "HOLIDAY"],
    variable_groups={},  # group of categorical variables can be treated as one variable
    time_varying_known_reals=["Quarter", "time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "TOTALDEMAND",
        "TEMPERATURE",
        "POWER"
    ],
    target_normalizer=GroupNormalizer(
        groups=["Month", "Year"], transformation="softplus"
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)